## Settings

### DC power supply

In [1]:
import pyvisa

#Create Resource manager instance for DC power supply
rm= pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('ASRL7::INSTR',)

In [9]:
# Start the communication by VISA identifying resource name
rnd = rm.open_resource('ASRL7::INSTR')

#Checking the machine name
print(rnd.query('*IDN?'))

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

### Definition of Stabilized Power Supply Operating Functions

In [10]:
#通信インスタンスとチャンネルと電圧と電流を指定して出力オン
def out_put_on(sps,ch,vol,cur):
    sps.write("VSET"+str(ch)+":"+str(vol)) #set ch1 output current 0.01V
    sps.write("ISET"+str(ch)+":"+str(cur)) #set ch1 output current 0.01A
    sps.write("OUT1")

def out_put_on_only_V(sps,ch,vol):
    sps.write("VSET"+str(ch)+":"+str(vol)) #set ch1 output current 0.01V
    sps.write("ISET"+str(ch)+":0") #set ch1 output current 0A
    sps.write("OUT1")

#通信インスタンスとチャンネル番号を指定して出力オフ
def out_put_off(sps,ch):
    sps.write("VSET"+str(ch)+":0") 
    sps.write("ISET"+str(ch)+":0") 
    sps.write("OUT1")

def out_put_off_all(sps):
    sps.write("VSET1:0") 
    sps.write("ISET1:0") 
    sps.write("VSET2:0") 
    sps.write("ISET2:0") 
    sps.write("OUT1")

### Teensy4.0

In [4]:
import serial.tools.list_ports

# COM port setting of Teensy4.0

# get available port list
ports = serial.tools.list_ports.comports()
portList = []

for one in ports:
    portList.append(str(one))
    print(str(one))

# select COM port of Teensy: Serielles USB-Gerät
com = input("Select Com Port for Teensy4.0 #:")

use = None
for i in range(len(portList)):
    if portList[i].startswith("COM" + str(com)):
        print(portList[i])
        use = "COM" + str(com)
        print(use)
        break

COM7 - Serielles USB-Gerät (COM7)
COM3 - Intel(R) Active Management Technology - SOL (COM3)
COM7 - Serielles USB-Gerät (COM7)
COM7


In [2]:
# Portrate setting
baudrate = 115200

if use is None:
    print("Selected COM port not found.")
else:
    # Make serial port instance
    try:
        serialInst = serial.Serial(use, baudrate, timeout=1) 
        print("Serial port opened successfully.")
    except serial.SerialException as e:
        print(f"Error opening serial port: {e}")
        exit()


Serial port opened successfully.


## Adjustment move

### Home (Please not use: Limit Switch is not working now)

In [3]:
#serialInst.write(('H').encode('utf-8'))

### X-axis (Enter arbitrary distance [mm])

In [4]:
displacement = input("X axis Distance (mm)")
serialInst.write(('X' + displacement).encode('utf-8'))

1

### Y-axis (Enter arbitrary distance [mm])

In [5]:
displacement = input("Y axis Distance (mm)")
serialInst.write(('Y' + displacement).encode('utf-8'))

1

### Z-axis (Enter arbitrary distance [mm])

In [6]:
displacement = input("Z axis Distance (mm)")
serialInst.write(('Z' + displacement).encode('utf-8'))

1

## Measurement

### DC power supply

In [ ]:
#DCDC converter input active voltage
out_put_on(rnd,1,0,0.1) # channel 1, 0 V, 0.1 A

### Stepping motor move

In [3]:
import csv
import os
import time
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Save folder
folder_path = "C:\\Users\\kary\\Documents\\9_data\\Electromagnet\\" # change path for yourself
file_path = os.path.join(folder_path, "position_data.csv")

# Initialize CSV file
def initialize_csv():
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "High_Precision_Timestamp", "X_Position", "Y_Position"])

# 位置情報のCSVファイルへの保存
def save_position_to_csv(x, y):
    now = datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
    high_precision_time = time.perf_counter()
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([now, high_precision_time, x, y])

# メッシュ設定
x_mesh_size = 1.0  # mm
y_mesh_size = 1.0  # mm
resolution = 0.5  # mm
x_num_points = int(x_mesh_size / resolution + 1)
y_num_points = int(y_mesh_size / resolution + 1)

def move_motor(axis, displacement):
    command = axis + str(displacement)
    serialInst.write(command.encode('utf-8'))
    # モーターが動作するための待機時間（調整が必要）
    time.sleep(1)

def run_mesh_movement():
    current_x = 0.0
    current_y = 0.0
    initialize_csv()
    save_position_to_csv(current_x, current_y)  # 初期位置 (0, 0) を記録

    for y in range(y_num_points):
        for x in range(x_num_points):
            target_y = y * resolution
            target_x = x * resolution

            # Y軸の相対位置を計算
            if target_y != current_y:
                move_motor('Y', target_y - current_y)
                current_y = target_y
            
            # X軸の相対位置を計算
            if target_x != current_x:
                move_motor('X', target_x - current_x)
                current_x = target_x

            # 物体が位置に到達するのを待つ（調整が必要）
            time.sleep(1)

            # シリアルポートからデータを読み取る
            try:
                if serialInst.in_waiting > 0:
                    data = serialInst.readline().decode('utf-8').strip()
                    if data == "Stopped":
                        save_position_to_csv(current_x, current_y)
                        print(f"Received data: {data} at X: {current_x}, Y: {current_y}")
                    else:
                        print(f"Unexpected data: {data}")
                else:
                    print("No data received.")
            except serial.SerialException as e:
                print(f"Error reading data: {e}")

        # 行の終わりで次の行に移動する前にX軸をリセットする
        if y != y_num_points - 1:
            move_motor('X', -current_x)
            current_x = 0.0

# ボタンがクリックされたときのイベントを処理する関数
def on_click(event):
    clear_output(wait=True)
    print("Starting the external program...")
    run_mesh_movement()
    serialInst.close()
    print("Program finished and serial port closed.")

# ボタンを作成
start_button = widgets.Button(description="Click here to start the program", layout=widgets.Layout(width='100%', height='100%'))
start_button.style.button_color = 'lightblue'

# ボタンクリックイベントを設定
start_button.on_click(on_click)

# Display Button
display(start_button)


Starting the external program...
No data received.
Received data: Stopped at X: 0.5, Y: 0.0
Received data: Stopped at X: 1.0, Y: 0.0
Received data: Stopped at X: 0.0, Y: 0.5
Received data: Stopped at X: 0.5, Y: 0.5
Received data: Stopped at X: 1.0, Y: 0.5
Received data: Stopped at X: 0.0, Y: 1.0
Received data: Stopped at X: 0.5, Y: 1.0
Received data: Stopped at X: 1.0, Y: 1.0
Program finished and serial port closed.


### Voltage OFF

In [ ]:
# Power off all
out_put_off_all(rnd)